# **常見訓練設定**
此份程式碼將會介紹隨著訓練過程，可以調整或者紀錄的函式。

## 本章節內容大綱
* ### EarlyStopping（已於 part3/2_Overfitting.ipynb 介紹）
* ### [ModelCheckpoint](#ModelCheckpoint)
* ### [LearningRateSchedular](#LearningRateSchedular)
* ### [CSVLogger](#CSVLogger)

## 匯入套件

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Tensorflow 相關套件
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks

## 創建資料集／載入資料集（Dataset Creating / Loading）

In [ ]:
# 上傳資料
!wget -q https://github.com/TA-aiacademy/course_3.0/releases/download/DL/Data_part3.zip
!unzip -q Data_part3.zip

In [ ]:
train_df = pd.read_csv('./Data/News_train.csv')
test_df = pd.read_csv('./Data/News_test.csv')

In [ ]:
train_df.head()

In [ ]:
X_df = train_df.iloc[:, :-1].values
y_df = train_df.y_category.values

In [ ]:
X_test = test_df.iloc[:, :-1].values
y_test = test_df.y_category.values

## 資料前處理（Data Preprocessing）

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
# Feature scaling
sc = StandardScaler()
X_scale = sc.fit_transform(X_df, y_df)
X_test_scale = sc.transform(X_test)

In [ ]:
# Convert to One-Hot encoding
y_onehot = keras.utils.to_categorical(y_df)
y_test_onehot = keras.utils.to_categorical(y_test)

In [ ]:
# train, valid/test dataset split
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_scale, y_onehot,
                                                      test_size=0.2,
                                                      random_state=17,
                                                      stratify=y_df)

In [ ]:
print(f'X_train shape: {X_train.shape}')
print(f'X_valid shape: {X_valid.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'y_valid shape: {y_valid.shape}')

## 模型建置（Model Building）

In [ ]:
def build_model(input_shape, output_shape):
    keras.backend.clear_session()
    tf.random.set_seed(17)

    model = keras.models.Sequential()
    model.add(layers.Dense(64,
                           input_shape=input_shape,
                           activation='tanh'))
    model.add(layers.Dense(64,
                           activation='tanh'))
    model.add(tf.keras.layers.Dense(output_shape,
                                    activation='softmax'))
    return model

<a name="ModelCheckpoint"></a>
* ## ModelCheckpoint

In [ ]:
model = build_model(X_train[0].shape, y_onehot.shape[1])
model.summary()

In [ ]:
model.compile(optimizer='nadam',
              loss='categorical_crossentropy',
              metrics=['acc'])

In [ ]:
model_path = './Data/callbacks_model.h5'  # 模型儲存的位置

# 建立 Checkpoint
checkpoint = callbacks.ModelCheckpoint(
        model_path,
        verbose=1,
        monitor='val_acc',    # 儲存模型的指標
        save_best_only=True,  # 是否只儲存最好的
        mode='max')           # 與指標搭配模式

In [ ]:
history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=20,
                    validation_data=(X_valid, y_valid),
                    callbacks=[checkpoint])

<a name="LearningRateSchedular"></a>
* ## LearningRateSchedular

In [ ]:
def schedule(epoch):  # 定義 learning rate 根據 epoch 要如何變動
    if epoch < 10:
        return 0.001
    elif epoch < 15:
        return 0.0001
    else:
        return 0.00001


# 建立 LearningRateScheduler
lr_schedule = callbacks.LearningRateScheduler(
    schedule, verbose=2)

In [ ]:
rlp = callbacks.ReduceLROnPlateau(
    monitor='val_loss',  # 是否進步的指標
    factor=0.1,  # 以 factor 的倍數調整 learning rate
    patience=5,  # 經過 patience 次沒有進步調整 learning rate
    verbose=2,
    mode='min')

In [ ]:
# 建立兩個 list 記錄選用不同 learing rate schedular 的訓練結果
train_loss_list = []
train_acc_list = []

# 建立兩個 list 記錄選用不同 learning rate schedular 的驗證結果
valid_loss_list = []
valid_acc_list = []

callback_l = {'non': [], 'lr_s': lr_schedule, 'rlp': rlp}
for cb in callback_l:
    print('Training a model with callbacks: {}'
          .format(cb))
    model = build_model(X_train[0].shape, y_onehot.shape[1])
    model.compile(optimizer='nadam',
                  loss='categorical_crossentropy',
                  metrics=['acc'])
    history = model.fit(X_train, y_train,
                        epochs=20,
                        batch_size=64,
                        validation_data=(X_valid, y_valid),
                        callbacks=[callback_l[cb]],
                        verbose=0)

    # 將訓練過程記錄下來
    train_loss_list.append(history.history['loss'])
    valid_loss_list.append(history.history['val_loss'])
    train_acc_list.append(history.history['acc'])
    valid_acc_list.append(history.history['val_acc'])
    print('\n')
print('----------------- training done! -----------------')

In [ ]:
# 視覺化訓練過程
plt.figure(figsize=(15, 5))

train_line = ()
valid_line = ()

# 繪製 Training loss
plt.subplot(121)
for k, cb in enumerate(callback_l):
    loss = train_loss_list[k]
    val_loss = valid_loss_list[k]
    train_l = plt.plot(
        range(len(loss)), loss,
        label=f'Training    callback:{cb}')
    valid_l = plt.plot(
        range(len(val_loss)), val_loss, '--',
        label=f'Validation callback:{cb}')

    train_line += tuple(train_l)
    valid_line += tuple(valid_l)
plt.title('Loss')

# 繪製 Training accuracy
plt.subplot(122)
train_acc_line = []
valid_acc_line = []
for k, cb in enumerate(callback_l):
    acc = train_acc_list[k]
    val_acc = valid_acc_list[k]
    plt.plot(range(len(acc)), acc,
             label=f'Training    callback:{cb}')
    plt.plot(range(len(val_acc)), val_acc, '--',
             label=f'Validation callback:{cb}')
plt.title('Accuracy')

first_legend = plt.legend(handles=train_line,
                          bbox_to_anchor=(1.05, 1))

plt.gca().add_artist(first_legend)
plt.legend(handles=valid_line,
           bbox_to_anchor=(1.05, 0.7))

<a name="CSVLogger"></a>
* ## CSVLogger

In [ ]:
model = build_model(X_train[0].shape, y_onehot.shape[1])
model.summary()

In [ ]:
model.compile(optimizer='nadam',
              loss='categorical_crossentropy',
              metrics=['acc'])

In [ ]:
csv_logger = callbacks.CSVLogger('./Data/training_log.csv')

In [ ]:
history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=20,
                    validation_data=(X_valid, y_valid),
                    callbacks=[csv_logger])

---
wandb（補充教材）: https://docs.wandb.ai/v/zh-hans/quickstart